In [97]:
# imports
import jax
from jaxopt import GradientDescent
import numpy as np

from defaults import *
from solver_runner import run_solver

In [98]:
class QuadraticProblem:
    """
    A class describing an unconstrained quadratic problem:
    0.5 x^TAx^T + b^Tx, where the matrix A is positive defined, x \in R^n
    """
    n: int = 1           # problem dimensionality
    A = None             # A-matrix: np.array of shape (n,n)
    b = None             # b-vector: np.array of shape (n,) 
    def __init__(self, n: int = 2, A = None, b = None):
        np.random.seed(default_seed)
        self.n = n
        if A is None:
            self.A = self.__get_random_matrix(self.n)
        else:
            self.A = A
        if b is None:
            self.b = self.__get_random_vector(self.n)
        else:
            self.b = b
        
    
    def f(self, x):
        """
        Target function
        x: np.array of shape (n, )
        return: float
        """
        return 0.5 * x.T @ self.A @ x + self.b.T @ x
    
    def grad_f(self, x):
        """
        Gradient of target function
        x: np.array of shape (n,)
        return: np.array of shape (n,)
        """
        return self.A.T @ x + self.b
    
    def fun(self, x):
        """
        x: np.array of shape (n,)
        return: tuple (f(x), grad_f(x))
        """
        return (self.f(x), self.grad_f(x))
    
    def __get_random_matrix(self, n: int = 2):
        """
        Returns a positive defined matrix of size (n, n)
        """
        B = np.random.rand(n, n)
        while np.linalg.det(B) == 0:
            B = np.random.rand(n, n)
        return B.T @ B

    def __get_random_vector(self, n: int = 2):
        """
        Returns a random vector: np.array of shape (n,)
        """
        return np.random.rand(n)

In [99]:
n = 2
qp = QuadraticProblem(n=n)
#print(qp.f(np.array([1., 1.])))
print(qp.A, qp.b)

[[0.67609655 0.62907433]
 [0.62907433 0.58994988]] [0.37223648 0.0304395 ]


In [117]:
class Benchmark:
    """
    A class that provides the benchmarking of different optimization methods on a quadratic problem (convex).
    """
    x_init = None              # initial point: np.array of shape (n,)
    def __init__(self, problem: QuadraticProblem = None, x_init = None):
        if problem is None:
            self.problem = QuadraticProblem(n=default_dimensionality)
        else:
            self.problem = problem
        
        if x_init is None:
            self.x_init = np.random.rand(problem.n)
        else:
            self.x_init = x_init
        
    
    def compare_via_nit(self):
        """
        Return: dict{key : value}
        key (str): the name of method
        value (int): number of iterations
        """
        # test for Gradient Descent with constant stepsize
        const_stepsize = 1e-2
        max_iters = 4000
        tolerance = 1e-4
        result = dict()
        gd = GradientDescent(
            fun=self.problem.fun,
            value_and_grad=True,
            has_aux=False,
            stepsize=const_stepsize,
            maxiter=max_iters,
            tol=tolerance,
            acceleration=False,
            verbose=0,
            implicit_diff=False
        )
        trajectory, errors = run_solver(gd, self.x_init)
        d = dict()
        d['GradientDescent'] = trajectory.shape[0]
        return d
    
    def compare_via_nfev(self):
        """
        Return: dict{key : value}
        key (str): the name of method
        value (int): number of function calls
        """
        pass
    
    def compare_via_njev(self):
        """
        Return: dict{key : value}
        key (str): the name of method
        value (int): number of gradient calls in this function
        """
        pass
    
    def comapre_via_time(self):
        """
        Return: dict{key : value}
        key (str): name of the method
        value (float): method's running time
        """

In [118]:
n = 2
qp = QuadraticProblem(n=n)
x_init = np.array([1, 10])
benchmark = Benchmark(problem=qp, x_init=x_init)
print(benchmark.compare_via_nit())
"""
params, state = gd.run(self.x_init)
print(f'params: {params}')
print(f'state: {state}')
"""

{'GradientDescent': 4000}


"\nparams, state = gd.run(self.x_init)\nprint(f'params: {params}')\nprint(f'state: {state}')\n"